In [25]:
import numpy as np
import matplotlib.pyplot as plt
import csv
import time 
import os


def clear(): return os.system('clear')

In [26]:
def printProgressBar(iteration, total, prefix='', suffix='', decimals=1, length=100, fill='█', printEnd="\r"):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
        printEnd    - Optional  : end character (e.g. "\r", "\r\n") (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 *
                                                     (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end=printEnd)
    # Print New Line on Complete
    if iteration == total:
        print()
    


def cross_validation_split(cross_validate_num, dataset):
    cross_len = int(round((len(dataset) * cross_validate_num), 0))
    block = []
    random_set = []
    reminder_set = []
    for i in range(10):
        rand = np.random.randint(0, 10)
        while random_set.__contains__(rand):
            rand = np.random.randint(0, 10)
        random_set.append(rand)
        block.append(dataset[i * cross_len:cross_len + (i * cross_len)])
        if i == 9 and sum([len(b) for b in block]) < len(dataset):
            reminder_set = dataset[cross_len +
                                   (i * cross_len):len(dataset)]
    return {
        "data_block": block,
        "rand_set": random_set,
        "rem_set": reminder_set
    }


def select_validate(block, random_set, c, rem_set):
    cross_valid = block[random_set[c]]
    train_idx = random_set.copy()
    train_idx.remove(random_set[c])
    train = []
    for n in range(9):
        train += block[train_idx[n]]
        if c == 9 and n == 8:
            train += rem_set
    return {
        "train": train,
        "cross_valid": cross_valid
    }

In [27]:
def sigmoid(x):
    return 1 / (1 + np.e**(-x))

class Node:
    def __init__(self):
        self.w = []
        self.b = 0.0
        self.input = []
        self.y = 0

    def initialWeight(self, n):
        self.w = np.random.uniform(-10, 10, size=n).reshape(n, 1)
        # self.b = np.random.uniform(-1, 1)

    def addInput(self, inputVector):
        self.input = inputVector

    def calculate_output(self, net):
        if len(net) != 0 and len(self.w) != 0:
            self.y = sigmoid(np.array(net).dot(self.w)[0] + self.b)
        else:
            self.y = np.array(self.input)

def buildLayer(node):
    return [Node() for i in range(node)]
    
class Model:
    def __init__(self):
        self.particle = []
        self.x = []
        self.pbest = float('inf')
        self.xpbest = []
        self.v = []

    def feed_forward(self, inputVector):
        for l, layer in enumerate(self.particle):
            for i, node in enumerate(layer):
                if l == 0:
                    node.addInput(inputVector[i])
                    node.calculate_output([])
                else:
                    net = [prev.y for prev in self.particle[l-1]]
                    node.calculate_output(net)

    def create(self, inputLayer, hiddenLayers, outputLayer):
        self.inputLayer = inputLayer
        self.hiddenLayers = hiddenLayers
        self.outputLayer = outputLayer

        self.particle.append(self.inputLayer)

        for layer in self.hiddenLayers:
            self.particle.append(layer)
        self.particle.append(self.outputLayer)

        for node in self.outputLayer:
            node.initialWeight(
                n=len(self.hiddenLayers[len(self.hiddenLayers) - 1]))
            for w in node.w:
                self.x.append(w)

        for l, layer in enumerate(self.hiddenLayers):
            for node in layer:
                if l == 0:
                    n = len(self.inputLayer)
                else:
                    n = len(self.hiddenLayers[l-1])
                node.initialWeight(n)
                for w in node.w:
                    self.x.append(w)

        self.x = np.array(self.x)
        self.v = np.random.uniform(-0.5, 0.5,
                                   size=len(self.x)).reshape(len(self.x), 1)

    def updateNeuralNetwork(self):
        count = 0
        for node in self.outputLayer:
            for i in range(len(node.w)):
                node.w[i] = self.x[count]
                count = count+1
        for layer in self.hiddenLayers:
            for node in layer:
                for i in range(len(node.w)):
                    node.w[i] = self.x[count]
                    count = count+1

    def evaluate(self, data):
        predict = 120
        MAE = []
        for n, train in enumerate(data):
            if n + predict <= len(data) - 1:
                self.feed_forward(train['input'])
                for node in self.outputLayer:
                    MAE.append(
                        np.abs(node.y - data[n+predict]['desire_output']))
        mean_absolute_err = np.average(MAE)
        return mean_absolute_err

In [28]:
def preprocessData():
    file = open('AirQualityUCI.csv', 'r')
    reader = csv.reader(file)
    data = []
    preData = {
        "3": [],
        "6": [],
        "8": [],
        "10": [],
        "11": [],
        "12": [],
        "13": [],
        "14": [],
        "desire_output": []
    }
    inputIdx = [3, 6, 8, 10, 11, 12, 13, 14]
    for i, row in enumerate(reader):
        if i > 0:
            preData['desire_output'].append(float(row[5]))
            for idx in inputIdx:
                preData[f'{idx}'].append(float(row[idx]))
    file.seek(0)
    maxOutput = np.max(preData['desire_output'])
    meanOutput = np.average(preData['desire_output'])
    inputVal = {
        "3": {"max": 0, "mean": 0},
        "6": {"max": 0, "mean": 0},
        "8": {"max": 0, "mean": 0},
        "10": {"max": 0, "mean": 0},
        "11": {"max": 0, "mean": 0},
        "12": {"max": 0, "mean": 0},
        "13": {"max": 0, "mean": 0},
        "14": {"max": 0, "mean": 0},
    }
    for idx in inputIdx:
        inputVal[f'{idx}']['max'] = np.max(preData[f'{idx}'])
        inputVal[f'{idx}']['mean'] = np.average(preData[f'{idx}'])
    for i, row in enumerate(reader):
        printProgressBar(i, 9358, 'preproceeing...', '', length=50)
        if i > 0:
            if float(row[5]) == -200:
                row[5] = meanOutput
            dic = {
                "date": row[0],
                "time": row[1],
                "input": [],
                "desire_output": float(row[5]) / maxOutput
            }

            for idx in inputIdx:
                if float(row[idx]) == -200:
                    row[idx] = inputVal[f'{idx}']['mean']
                dic['input'].append(
                    float(row[idx]) / inputVal[f'{idx}']['max'])

            data.append(dic)
    return data


def initPopulation(n):
    population = []
    print("initing...")
    for p in range(n):
        inputLayer = buildLayer(node=8)
        hiddenLayers = [buildLayer(node=4), buildLayer(node=3)]
        outputLayer = buildLayer(node=1)
        print(f'p : {p}', end=" ")
        particle = Model()
        particle.create(inputLayer, hiddenLayers, outputLayer)
        print(
            f'shape :{len(particle.inputLayer)} - {[len(l) for l in particle.hiddenLayers]} - {len(particle.outputLayer)}')
        population.append(particle)
    return population


def velocity_vector(xpbest, xi, vi):
    return vi + (np.random.uniform(0, 1)*(xpbest - xi))

In [29]:
data = preprocessData()

t_max = 20
particle_n = 10

cross_data = cross_validation_split(cross_validate_num=0.1, dataset=data)
block = cross_data["data_block"]
rand_set = cross_data["rand_set"]
reminder_set = cross_data["rem_set"]

first_population = initPopulation(particle_n)
time.sleep(3)
cross_validation_plot = []
train_plot = []
wins = []

preproceeing... |██████--------------------------------------------| 12.2% 

preproceeing... |███████-------------------------------------------| 15.5% 

preproceeing... |███████████████-----------------------------------| 31.6% 

preproceeing... |███████████████████████---------------------------| 47.6% 

preproceeing... |██████████████████████████████████----------------| 68.9% 

preproceeing... |██████████████████████████████████████████████----| 93.7% 

initing...
p : 0 shape :8 - [4, 3] - 1
p : 1 shape :8 - [4, 3] - 1
p : 2 shape :8 - [4, 3] - 1
p : 3 shape :8 - [4, 3] - 1
p : 4 shape :8 - [4, 3] - 1
p : 5 shape :8 - [4, 3] - 1
p : 6 shape :8 - [4, 3] - 1
p : 7 shape :8 - [4, 3] - 1
p : 8 shape :8 - [4, 3] - 1
p : 9 shape :8 - [4, 3] - 1


In [30]:
for c in range(10):
    res = select_validate(block, rand_set, c, reminder_set)
    train = res["train"]
    cross_valid = res["cross_valid"]
    population = first_population
    printProgressBar(
        0, t_max, prefix=f'evolutioning.. t={1} ,cross_validation : {c+1}', length=25)
    tic = time.perf_counter()
    for t in range(t_max):

        printProgressBar(0, len(population), prefix='evaluating...', length=25)
        for i, p in enumerate(population):
            fx = p.evaluate(train)  # evaluate performance
            if fx < p.pbest:
                p.pbest = fx
                p.xpbest = p.x
                wins.append(p.pbest)
            # velocity calc
            v_new = velocity_vector(xpbest=p.xpbest, xi=p.x, vi=p.v)
            p.v = v_new
            # update position
            p.x = p.x + v_new
            # update weight with new position xi
            p.updateNeuralNetwork()
            printProgressBar(i+1, len(population),
                             prefix='evaluating...', suffix=f'p({i+1})fx = {fx} pbest : {p.pbest}', length=25)
       
        if t + 1 == t_max:
            temp = []
            for p in population:
                temp.append(p.pbest)
            train_plot.append(np.min(temp))
            
        clear()
        printProgressBar(c, 10, prefix=f'overall process : cross validation : {c+1} crossValidation : {cross_validation_plot}',
                         length=50, printEnd='\n')
        np.random.shuffle(train)
        printProgressBar(
            t+1, t_max, prefix=f'evolutioning.. t={t+1}', suffix=f'best = {np.min(wins)}', length=25, printEnd='\n')
    # cross validation
    printProgressBar(0, len(population),
                     prefix='cross validationing...', length=25)
    best_validation = []
    for i, p in enumerate(population):
        mae = p.evaluate(cross_valid)
        best_validation.append(mae)
        printProgressBar(i+1, len(population),
                         prefix='cross validationing...', length=25)
    cross_validation_plot.append(np.min(best_validation))
    clear()
    toc = time.perf_counter()
    print(f'pbest in all gen = {np.min(wins)}')
    if toc-tic >= 60:
        print(f"total execution time {((toc - tic)/60):0.2f} m\n")
    else:
        print(f"total execution time {toc - tic:0.4f} secs\n")

    printProgressBar(c+1, 10, prefix=f'overall process : cross validation : {c+1} crossValidation : {cross_validation_plot}',
                     length=50, printEnd='\n')

plt.plot(cross_validation_plot,label='Cross validation test')
plt.plot(train_plot, label='Train')
print(cross_validation_plot)
print(train_plot)
plt.show()


0.34609186169699496
0.8424482893134829
0.8421656682175261
0.6787199667529299
0.15517271616135925
0.6698023539655087
0.8352867840419674
0.13863860425380597
0.8388297456071155
0.13781753397171398
0.8423117988844593
0.8420993744229089
0.14779086048264164
0.5861954089816045
0.8314182925923818
0.125653887591507
0.14083672082132226
0.5059787297649507
0.8256699912276129
0.8421027914704656
0.8419505399436912
0.13425526876186014
0.44266480179540746
0.8161966547667641
0.3460325231229641
0.676384800034034
0.12470081949060813


KeyboardInterrupt: 